In [ ]:
# Cell 1: Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Cell 2: Install necessary libraries
!pip install moviepy opencv-python bm3d cupy

# Cell 3: Import libraries
import numpy as np
import moviepy.editor as mp
import cv2  # Add import statement for OpenCV
import cupy as cp  # Import CuPy for GPU acceleration
from bm3d import bm3d_cpu, bm3d_gpu  # Import BM3D denoising functions for CPU and GPU

# Cell 4: Define function for spatial filtering (Gaussian blur)
def spatial_filter(frame, kernel_size=5, sigma=1):
    blurred_frame = cv2.GaussianBlur(frame, (kernel_size, kernel_size), sigma)
    return blurred_frame

# Cell 5: Specify directory where the video is located
video_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/un-preprocessed videos/'

# Cell 6: Specify the name of the input video file
video_filename = 'noisy building panning.mp4'

# Cell 7: Process and denoise the video in segments with reduced frame size
video_path = video_directory + video_filename

video_clip = mp.VideoFileClip(video_path)
fps = video_clip.fps
duration = video_clip.duration
segment_duration = 10  # Process 10 seconds of video at a time
num_segments = int(np.ceil(duration / segment_duration))

output_frames_spatial = []
output_frames_bm3d = []

def process_segment(segment_clip):
    segment_clip_resized = segment_clip.resize((1366, 768))  # Resize frame to reduce memory usage
    segment_frames = [frame for frame in segment_clip_resized.iter_frames()]

    # Apply spatial filtering (Gaussian blur)
    denoised_frames_spatial = [spatial_filter(frame) for frame in segment_frames]

    # Apply BM3D denoising with GPU acceleration
    denoised_frames_bm3d = [bm3d_gpu(cp.array(frame), sigma_psd=5) for frame in segment_frames]  # Adjust sigma_psd if needed

    return denoised_frames_spatial, [frame.get() for frame in denoised_frames_bm3d]  # Convert back to CPU memory

# Process segments in parallel
results = Parallel(n_jobs=-1)(delayed(process_segment)(video_clip.subclip(i * segment_duration, min((i + 1) * segment_duration, duration))) for i in range(num_segments))

# Extract denoised frames from results
for denoised_frames_spatial, denoised_frames_bm3d in results:
    output_frames_spatial.extend(denoised_frames_spatial)
    output_frames_bm3d.extend(denoised_frames_bm3d)

# Cell 8: Specify directory where the denoised videos will be saved
output_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/'

# Cell 9: Create a new video clip with spatially denoised frames
denoised_video_clip_spatial = mp.ImageSequenceClip(output_frames_spatial, fps=fps)

# Cell 10: Define output video file path for spatially denoised video
output_video_path_spatial = output_directory + 'building_panning_spatial_denoised_video.mp4'

# Cell 11: Write spatially denoised video clip to the output file
denoised_video_clip_spatial.write_videofile(output_video_path_spatial)

# Cell 12: Provide download link for the spatially denoised video
print("Spatial denoising completed. Download the spatially denoised video:")
print(output_video_path_spatial)

# Cell 13: Create a new video clip with BM3D denoised frames
denoised_video_clip_bm3d = mp.ImageSequenceClip(output_frames_bm3d, fps=fps)

# Cell 14: Define output video file path for BM3D denoised video
output_video_path_bm3d = output_directory + 'building_panning_bm3d_denoised_video.mp4'

# Cell 15: Write BM3D denoised video clip to the output file
denoised_video_clip_bm3d.write_videofile(output_video_path_bm3d)

# Cell 16: Provide download link for the BM3D denoised video
print("BM3D denoising completed. Download the BM3D denoised video:")
print(output_video_path_bm3d)


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
ERROR: Could not build wheels for cupy, which is required to install pyproject.toml-based projects


ImportError: 
================================================================
Failed to import CuPy.

If you installed CuPy via wheels (cupy-cudaXXX or cupy-rocm-X-X), make sure that the package matches with the version of CUDA or ROCm installed.

On Linux, you may need to set LD_LIBRARY_PATH environment variable depending on how you installed CUDA/ROCm.
On Windows, try setting CUDA_PATH environment variable.

Check the Installation Guide for details:
  https://docs.cupy.dev/en/latest/install.html

Original error:
  ImportError: libcuda.so.1: cannot open shared object file: No such file or directory
================================================================
